In [2]:
import sympy
from sympy import *
import math

def properties(poly):
    x = symbols('x')
    polinL = poly.split(',')
    if len(polinL) == 1:
        pol = polinL[0]
        pol= Poly(pol, x, domain= 'QQ')
        print(pol.rep.rep)
        if len(pol.rep.rep) == 1:
            polLin=pol.rep.rep
            raices = polLin[0] if polLin[0] == 0 else []
            deriv1= Poly(0, x, domain= 'QQ')
            raicesD1 = [0]
            raicesyDer = [0]
            deriv2= Poly(0, x, domain= 'QQ')
            raicesD2 = [0]
            ventanaX=[-10, 10]
        elif len(pol.rep.rep) == 2:
            polLin = pol.rep.rep
            raices = [float(-polLin[1]/polLin[0])]
            deriv1 = diff(pol, x)
            deriv2 = diff(deriv1, x)
            raicesD1 = []
            raicesyDer = raices.copy()
            raicesD2 = [0]
            ventanaX = [floor( -abs(raices[0])*2.0), math.ceil( abs(raices[0])*2.0)]
        elif len(pol.rep.rep) == 3:
            raices = aprox(real_roots(pol))
            deriv1 = diff(pol, x)
            deriv2 = diff(deriv1, x)
            polDer= deriv1.rep.rep
            raicesD1 = [float(-polDer[1]/polDer[0])]
            raicesyDer = raices + raicesD1
            raicesyDer.sort()
            raicesD2 = []
            if len(raicesyDer) > 1:
                ventanaX = [1.1*raicesyDer[0], 1.1*raicesyDer[-1]]
            else:
                ventanaX= [- abs(raicesyDer[0])*2, abs(raicesyDer[0])*2]
            ventanaX[0]= floor(ventanaX[0])
            ventanaX[1]= math.ceil(ventanaX[1])
        else:    
            deriv1 = diff(pol, x)
            deriv2 = diff(deriv1, x)
            pol = Poly(pol)
            raices = aprox(real_roots(pol))
            raicesD1 = aprox(real_roots(deriv1))
            der2Pol = Poly(deriv2)
            raicesD2 = aprox(real_roots(der2Pol))
            raicesyDer = raices + raicesD1
            raicesyDer.sort()
            ventanaX = [1.1*raices[0] - 0.1*raices[-1], 1.1*raices[-1] - 0.1*raices[0]] if (len(raices) > 1) \
                else [- abs(raices[0])*1.2, abs(raices[0])*1.2]
            ventanaX[0]= floor(ventanaX[0])
            ventanaX[1]= math.ceil(ventanaX[1])
        return {
            "racional": format(false),
            "polinomio": {
                "expr": format(pol.as_expr()),
                "latex": latex(pol.as_expr()),
            },
            "derivada": {
                "expr": format(deriv1.as_expr()),
                "latex": latex(deriv1.as_expr()),
            },
            "derivada2": { 
                "expr": format(deriv2.as_expr()),
                "latex": latex(deriv2.as_expr()),
            },
            "raices": {
                "rfun": format(raices),
                "rder1": format(raicesD1), 
                "rder2": format(raicesD2),
            },
            "ventanaX": format(ventanaX),
        }
    elif len(polinL) == 2:
        polNum, polDen = polinL
        polNum = Poly(polNum, x, domain= 'QQ')
        polDen = Poly(polDen,x, domain='QQ')
        raices = aprox(real_roots(polNum))
        polos = aprox(real_roots(polDen))
        remov = discontRemov(raices, polos)
        raices = quitaRemov(raices, remov)
        polos = quitaRemov(polos, remov)
        rac = polNum/polDen
        derNum = diff(polNum, x)*polDen-polNum*diff(polDen, x)
        deriv1 = rac.diff()
        junDeriv= together(deriv1)
        deriv2 = diff(deriv1, x)
        der2Num = diff(derNum, x)*polDen**2 - \
            2*polDen*diff(polDen, x)*derNum
        raicesD1 = aprox(real_roots(derNum))
        remov= discontRemov(raicesD1, polos)
        raicesD1 = quitaRemov(raicesD1, remov)
        raicesD2 = quitaRemov(aprox(real_roots(der2Num)), remov)
        polosRaicesyDer= polos + raices
        polosRaicesyDer= polosRaicesyDer + raicesD1
        polosRaicesyDer.sort()
        ventanaX = [1.1*polosRaicesyDer[0] - 0.1*polosRaicesyDer[-1],   1.1*polosRaicesyDer[-1] - 0.1*polosRaicesyDer[0]] if (len(polosRaicesyDer) > 1) \
            else [- abs(polosRaicesyDer[0])*1.2, abs(polosRaicesyDer[0])*1.2]
        ventanaX[0]= floor(ventanaX[0])
        ventanaX[1]= math.ceil(ventanaX[1])
        return {
            "racional": format(true),
            "polNum": {
                "expr": format(polNum.as_expr()),
                "latex": latex(polNum.as_expr())
            },
            "polDen": {
                "expr": format(polDen.as_expr()),
                "latex": latex(polDen.as_expr())
            },
            "derivada": {
                "expr": format(deriv1),
                "latex": latex(deriv1),
                "latexS": latex(junDeriv)
            },
            "derivada2": {
                "expr": format(deriv2),
                "latex": latex(deriv2)
            },
            "raices": {
                "rfun": format(raices),
                "rder1": format(raicesD1),
                "rder2": format(raicesD2)
            },
            "polos": format(polos),
            "remov": format(remov),
            "ventanaX": format(ventanaX),
            "polosRaicesyDer": format(polosRaicesyDer),
        }
    else:
        return {"error": "algo salio mal"}


def aprox(raices):
    rAprox = []
    for r in raices:
        if isinstance(r, Rational):
            rAprox.append(r)
        else:
            rAprox.append(r.evalf(15))
    return rAprox

def discontRemov(raices, polos):
    remov= []
    for r in raices:
        for p in polos:
            if r == p:
                remov.append(r)
    return remov

def quitaRemov(arr, remov):
    result= arr.copy()
    for item in remov:
        while True:
            try:
                result.remove(item)
            except ValueError:
                break        
    return result

properties("x**3-3*x+1,x**2-x**3")



{'racional': 'True',
 'polNum': {'expr': 'x**3 - 3*x + 1', 'latex': 'x^{3} - 3 x + 1'},
 'polDen': {'expr': '-x**3 + x**2', 'latex': '- x^{3} + x^{2}'},
 'derivada': {'expr': '(3*x**2 - 3)/(-x**3 + x**2) + (3*x**2 - 2*x)*(x**3 - 3*x + 1)/(-x**3 + x**2)**2',
  'latex': '\\frac{3 x^{2} - 3}{- x^{3} + x^{2}} + \\frac{\\left(3 x^{2} - 2 x\\right) \\left(x^{3} - 3 x + 1\\right)}{\\left(- x^{3} + x^{2}\\right)^{2}}',
  'latexS': '\\frac{3 x \\left(1 - x\\right) \\left(x^{2} - 1\\right) + \\left(3 x - 2\\right) \\left(x^{3} - 3 x + 1\\right)}{x^{3} \\left(1 - x\\right)^{2}}'},
 'derivada2': {'expr': '6*x/(-x**3 + x**2) + (6*x - 2)*(x**3 - 3*x + 1)/(-x**3 + x**2)**2 + 2*(3*x**2 - 3)*(3*x**2 - 2*x)/(-x**3 + x**2)**2 + (3*x**2 - 2*x)*(6*x**2 - 4*x)*(x**3 - 3*x + 1)/(-x**3 + x**2)**3',
  'latex': '\\frac{6 x}{- x^{3} + x^{2}} + \\frac{\\left(6 x - 2\\right) \\left(x^{3} - 3 x + 1\\right)}{\\left(- x^{3} + x^{2}\\right)^{2}} + \\frac{2 \\cdot \\left(3 x^{2} - 3\\right) \\left(3 x^{2} - 2 x\\right)